In [1]:
## system package
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os, sys
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="4"  # specify which GPU(s) to be used
sys.path.append('../Step4_Training/')
sys.path.append('../')

from pathlib import Path
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm_notebook as tqdm
from sklearn.metrics import cohen_kappa_score

from input.inputPipeline_stiching import data_transform, PandaPatchDataset, crossValDataloader
from utiles.radam import *
from utiles.utils import *
from utiles.flatten_cosanneal import *
from Model.efficientnet.model import EfficientNet as Model

In [2]:
fold = 1
portion = 60
mode = f"curriculum_easy_{portion}idx"
idxs = np.load(f"./selected_idx_{mode}_{fold}.npy", allow_pickle=True).item()
keys = ['exclude_noisy', 'include_expert', 'include_auto']

### Make Predictions

In [3]:
# csv_file = "../Data_CSV/Train_Data.csv"
csv_file = f"../Data_CSV/Train_Data_curriculum_easy_{portion}idx_{fold}.csv"
data_dir = '../../tile_concat_wy/input/panda-36x256x256-tiles-data-opt/train'
sz = 256
bs = 6
tsfm = None

In [4]:
model = Model.from_pretrained('efficientnet-b0', num_classes = 5).cuda()
model_path = f'../Step4_Training/weights/Resnext50_36patch_constant_lr_{mode}/Resnext50_36patch_constant_lr_{mode}_{fold}_best.pth.tar'
pretrained_dict = torch.load(model_path)
# pretrained_dict = pretrained_dict['state_dict']
model_dict = model.state_dict()
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
model_dict.update(pretrained_dict)
model.load_state_dict(pretrained_dict)

Loaded pretrained weights for efficientnet-b0


<All keys matched successfully>

In [5]:
model.eval()
for key in keys:
    dataset = PandaPatchDataset(csv_file, data_dir, sz, transform=tsfm, N = 36, rand = False)
    idx = idxs.get(key)
    dataset_sub = torch.utils.data.Subset(dataset, idx)
    dataloader = torch.utils.data.DataLoader(dataset_sub, batch_size = bs, shuffle = False, num_workers = 4, pin_memory=True)
    predictions = []
    img_idxs = []
    gth = []
    for i, data in enumerate(tqdm(dataloader)):
        with torch.no_grad():
            img, img_idx, grade = data['img'].cuda().float(), data['idx'], data['isup_grade']
            preds = model(img)
            preds = preds.sigmoid().sum(1).round().cpu()
            predictions.append(preds)
            img_idxs.append(img_idx)
            gth.append(grade.sum(1))

    predictions_ = torch.cat(predictions).numpy()
    img_idxs_ = torch.cat(img_idxs).numpy()
    gth_ = torch.cat(gth).numpy()
    kappa = cohen_kappa_score(predictions_, gth_, weights="quadratic")
    print(f"kappa score for {key}: {kappa}")


kappa score for exclude_noisy: 0.886754617414248



kappa score for include_expert: 0.6996625421822271



kappa score for include_auto: 0.7017322266329844


In [6]:
len(predictions_), len(img_idxs_)

(57, 57)

### Construct new csv file

In [7]:
df = pd.read_csv(csv_file)
df_temp = df.iloc[img_idxs_]
# df_temp["isup_grade"] = predictions_
df.to_csv(f"../Data_CSV/Train_Data_curriculum_easy_{portion + 10}idx_{fold}.csv")

### Construct new idx file

In [8]:
idx_in_pool = set(np.load(f"../Idxs/{mode}_{fold}.npy"))
idx = list(idx_in_pool.difference(set(idxs["exclude_noisy"])))
idx = list(set(idx + list(idxs["include_auto"]) + list(idxs["include_expert"])))
np.save(f"../Idxs/curriculum_easy_{portion+10}idx_{fold}.npy", np.asarray(idx))

In [9]:
len(idx)

7412